# In this we're trying to create a Synthetic Dataset of Analog Clock

In [1]:
%pwd

'd:\\Advanced Project\\time-estimation\\notebooks'

In [2]:
# Library
import math
import cv2
import datetime
import numpy as np
from pathlib import Path
import os
from timeEstimator.Utils.common import create_directory
import pandas as pd

In [3]:
os.chdir("../")

In [4]:
root_dir = Path('raw')
image_dir = os.path.join(root_dir, "images")

create_directory([root_dir, image_dir])

lable_csv = os.path.join(root_dir, "label.csv")

[2025-02-25 20:03:38,682]: INFO: common : Directory has been Created: raw
[2025-02-25 20:03:38,687]: INFO: common : Directory has been Created: raw\images


In [5]:
COLORS = {
    'black': (0,0,0)}

In [6]:
# Clock Geometry
CLOCK_RADIUS = 260
CENTER = (320, 320)
CANVAS_SIZE = (640, 640)

In [7]:
def get_ticks():
	hours_init = []
	hours_dest = []

	for i in range(0,360,6):
		x_coordinate = int(CENTER[0] + CLOCK_RADIUS * math.cos(i * math.pi / 180))
		y_coordinate = int(CENTER[1] + CLOCK_RADIUS * math.sin(i * math.pi / 180))

		hours_init.append((x_coordinate,y_coordinate))

	for i in range(0,360,6):
		x_coordinate = int(CENTER[0] + (CLOCK_RADIUS-20) * math.cos(i * math.pi / 180))
		y_coordinate = int(CENTER[1] + (CLOCK_RADIUS-20) * math.sin(i * math.pi / 180))

		hours_dest.append((x_coordinate,y_coordinate))

	return hours_init, hours_dest

In [8]:
def draw_time(image, hour, minute, second):

	hour =hour
	minute = minute
	second = second

	second_angle = math.fmod(second * 6 + 270, 360)
	minute_angle = math.fmod(minute * 6 + 270, 360)
	hour_angle = math.fmod((hour*30) + (minute/2) + 270, 360)

	second_x = int(CENTER[0] + (CLOCK_RADIUS-25) * math.cos(second_angle * math.pi / 180))
	second_y = int(CENTER[1] + (CLOCK_RADIUS-25) * math.sin(second_angle * math.pi / 180))
	cv2.line(image, CENTER, (second_x, second_y), COLORS['black'], 2)

	minute_x = int(CENTER[0] + (CLOCK_RADIUS-60) * math.cos(minute_angle * math.pi / 180))
	minute_y = int(CENTER[1] + (CLOCK_RADIUS-60) * math.sin(minute_angle * math.pi / 180))
	cv2.line(image, CENTER, (minute_x, minute_y), COLORS['black'], 3)

	hour_x = int(CENTER[0] + (CLOCK_RADIUS-100) * math.cos(hour_angle * math.pi / 180))
	hour_y = int(CENTER[1] + (CLOCK_RADIUS-100) * math.sin(hour_angle * math.pi / 180))
	cv2.line(image, CENTER, (hour_x, hour_y), COLORS['black'], 7)

	cv2.circle(image, CENTER, 5, COLORS['black'], -1)

	return image

In [9]:
image = np.zeros(CANVAS_SIZE, dtype=np.uint8)
image[:] = 255

hour_init, hour_end = get_ticks()
window_name = "Circle"

for i in range(len(hour_init)):
	if i % 5 == 0:
		cv2.line(image, hour_init[i], hour_end[i], COLORS['black'], 3)
		hour_label = 12 if (i // 5) == 0 else (i // 5)

		text_radius = CLOCK_RADIUS - 50
		angle_rad = math.radians(i * 6 + 270)
		text_x = int(CENTER[0] + text_radius * math.cos(angle_rad))
		text_y = int(CENTER[1] + text_radius * math.sin(angle_rad))
        
        
		text_size = cv2.getTextSize(str(hour_label), cv2.FONT_HERSHEY_SIMPLEX, 0.8, 2)[0]
		text_origin = (text_x - text_size[0] // 2, text_y + text_size[1] // 2)
        
        
		cv2.putText(image, str(hour_label), text_origin, cv2.FONT_HERSHEY_SIMPLEX, 0.8, COLORS['black'], 2)
	else:
		cv2.circle(image, hour_init[i], 5, COLORS['black'], -1)
cv2.circle(image, (320,320), CLOCK_RADIUS+10, COLORS['black'], 2)
img = cv2.circle(img = image, center = CENTER, radius = CLOCK_RADIUS, color = (255, 255, 255))


hours = np.arange(1, 13)
minutes = np.arange(0, 60)
seconds = np.arange(0, 60)

hr, mi, se = np.meshgrid(hours, minutes, seconds, indexing='ij')

format_time = np.vectorize(lambda h, m, s: f"{h:02d}:{m:02d}:{s:02d}")
time_strings = format_time(hr, mi, se)

time_strings_flat = time_strings.flatten()
df = pd.DataFrame(columns=["img_dir", "hour", "minute", "second"])


for i in range(len(time_strings_flat)):
		image_original = image.copy()

		time_obj = datetime.datetime.strptime(str(time_strings_flat[i]), "%I:%M:%S").time()
		hour = time_obj.hour
		minutes = time_obj.minute
		seconds = time_obj.second

		clock_face = draw_time(image=image_original, hour=hour, minute=minutes, second=seconds)

		filename = os.path.join(image_dir, f"{i}.png")

		new_df = pd.DataFrame({
			'img_dir': [filename],
			'hour': [hour],
			'minute': [minutes],
			'second': [seconds]
		})

		df = pd.concat([df, new_df], ignore_index=True)

		cv2.imwrite(filename=filename, img=image_original)

df.to_csv(lable_csv,index=False)

In [10]:
img_list = os.listdir("raw/images")

In [11]:
len(img_list)

43200

In [14]:
df = pd.read_csv("raw/label.csv")
df.head()

,img_dir,hour,minute,second
0,raw\images\0.png,1,0,0
1,raw\images\1.png,1,0,1
2,raw\images\2.png,1,0,2
3,raw\images\3.png,1,0,3
4,raw\images\4.png,1,0,4
